# Steps to getting an Authorization Token
1. Register a web application on MyAnimeList as described in https://myanimelist.net/blog.php?eid=835707
2. Store your MyAnimeList Client ID and Client Secret in the files `data/mal_authentication/clientid.txt` and `data/mal_authentication/clientsecret.txt`
3. run the notebook

In [1]:
import json
import os
import secrets

import requests

os.chdir("../../data/mal_authentication")
with open("clientid.txt") as f:
    CLIENT_ID = f.read().strip()
with open("clientsecret.txt") as f:
    CLIENT_SECRET = f.read().strip()

# 1. Generate a new Code Verifier / Code Challenge.
def get_new_code_verifier() -> str:
    token = secrets.token_urlsafe(100)
    return token[:128]


# 2. Print the URL needed to authorise your application.
def print_new_authorisation_url(code_challenge: str):
    global CLIENT_ID

    url = f"https://myanimelist.net/v1/oauth2/authorize?response_type=code&client_id={CLIENT_ID}&code_challenge={code_challenge}"
    print(f"Authorise your application by clicking here: {url}\n")
    print(
        f"After pressing 'Allow', it's okay if the page fails to open, you just"
        " need to copy the URL that it tries to redirect you to\n"
    )


# 3. Once you've authorised your application, you will be redirected to the webpage you've
#    specified in the API panel. The URL will contain a parameter named "code" (the Authorisation
#    Code). You need to feed that code to the application.
def generate_new_token(authorisation_code: str, code_verifier: str) -> dict:
    global CLIENT_ID, CLIENT_SECRET

    url = "https://myanimelist.net/v1/oauth2/token"
    data = {
        "client_id": CLIENT_ID,
        "client_secret": CLIENT_SECRET,
        "code": authorisation_code,
        "code_verifier": code_verifier,
        "grant_type": "authorization_code",
    }

    response = requests.post(url, data)
    response.raise_for_status()  # Check whether the requests contains errors

    token = response.json()
    response.close()
    print("Token generated successfully!")

    with open("token.json", "w") as file:
        json.dump(token, file, indent=4)
        print('Token saved in "token.json"')

    return token


# 4. Test the API by requesting your profile information
def print_user_info(access_token: str):
    url = "https://api.myanimelist.net/v2/users/@me"
    response = requests.get(url, headers={"Authorization": f"Bearer {access_token}"})

    response.raise_for_status()
    user = response.json()
    response.close()

    print(f"\n>>> Greetings {user['name']}! <<<")

In [2]:
code_verifier = code_challenge = get_new_code_verifier()
print_new_authorisation_url(code_challenge)

Authorise your application by clicking here: https://myanimelist.net/v1/oauth2/authorize?response_type=code&client_id=2c833385ced9d77b887078684b405fbd&code_challenge=GNFAgwRkr13g-UlPHVZY42i-I9CzqmCnTB4TKECyIWD4W8XYmWT-8VzyxJQ13VN3JzJfolFBhttB7iUME8TUwKg39QdvNowF6j-NkJkDMb24jv9dvXOK7OyNKZ4QzDQQ

After pressing 'Allow', it's okay if the page fails to open, you just need to copy the URL that it tries to redirect you to



In [3]:
authorisation_code = input("Copy-paste the Authorisation Code or URL: ").strip()
default_url_prefix = "http://localhost/oauth?code="
if authorisation_code.startswith(default_url_prefix):
    authorisation_code = authorisation_code[len(default_url_prefix) :]
token = generate_new_token(authorisation_code, code_verifier)

Copy-paste the Authorisation Code or URL:  http://localhost/oauth?code=def50200c7c65f6f7c933dd28383b3a15ba52a69757f9012ed8efe9d46af9faf1090521813d306962eb1cef87ca005a7fda99ae4083782505c834d40bce615401c94417cae88d6c48b833e8cf1850a2a7dfb4f903a9af92c012a3bf2ba07d172ebb6838473bad3cfa056f668a673413b9ced4014880e789702027b83a8fba7b20989660661efd92e96d1f0ed01abc154bf5440cd8e1e5ae97bff03e65fbba279119559992d49f6476e3ad7838ddd1d9da999b581fab6744fd8f41b3dc939c22e717963f22c987735a4b21b5c7285d21f3887559b4826351dd866a504ff78216eece2a0774f10e1e64f4ee86f79d2db6556ad96b4c3defb4405b33e4cddf8c464fb796cc0f1ee35fc4fee713020c1a1f27e5e8eac49a27a15e0ed7452c833a3cd94de1bd99e849d3f1c8f4d6d83fc0658a4da7d69dee81054d2d4c5b6c40f2070c4a2b08021f02fbf34017922ffcfa5fd575e5458a4b15c2f243f8074042c4230d3cc2791d570a0234f8407d6080576e1e22f86d59ed34369990451e9f5e99668ba70af47a3a444398f7b17d6efbe723712f6e317220c8b39c6b0405b356c13a3b820729ce90b31f7b6461aba948b581c3d498c8febc510c087dab4b465a710904050fb97c824461025fc7a16fadfbbfc1

Token generated successfully!
Token saved in "token.json"


In [4]:
print_user_info(token['access_token'])


>>> Greetings Fro116! <<<
